In [2]:
import io
import json
import os
import sys
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests

In [9]:
headers = {
    'Accept-Encoding': 'gzip, deflate, sdch',
    'Accept-Language': 'en-US,en;q=0.8',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Cache-Control': 'max-age=0',
    'Connection': 'keep-alive',
}

In [10]:
r = requests.get("https://www.letrot.com/fr/courses/calendrier-resultats", headers=headers)

In [12]:
soup = bs(r.text, "html.parser")

In [18]:
courses = soup.find_all("a", {"class": "racesHippodrome"})

In [30]:
courses[0].text[2:].replace(" ", "")

'VINCENNES'

In [31]:
courses[0].get("href")

'/stats/courses/programme/2022-11-03/7500'

In [70]:
def reunions(debut, fin):
    courses_dict = []
    url = f"https://www.letrot.com/fr/courses/calendrier-resultats?publish_up={debut}&publish_down={fin}"
    r = requests.get(url, headers=headers)
    soup = bs(r.text, "html.parser")
    courses_raw = soup.find_all("a", {"class": "racesHippodrome"})
    
    for c in courses_raw:
        date = c.get("href").split("/")[-2]
        courses_dict.append({"date": date, "idHippo": c.get("href").split("/")[-1], "Hippodrome": c.text[2:].replace(" ", ""), "lien": c.get("href")})
    
    return pd.DataFrame(courses_dict)

In [71]:
reu = reunions("01-11-2022", "04-11-2022")

In [49]:
r = requests.get("https://www.letrot.com/stats/courses/programme/2018-11-01/5305/json", headers=headers)
r.json()

{'toReload': False,
 'nomHippodrome': 'LAVAL',
 'course': [{'heureCourse': '12:20',
   'discipline': 'Attelé',
   'numCourse': 1,
   'prix': 'PRIX DU CREDIT AGRICOLE',
   'allocation': '22 000',
   'distance': '2 875',
   'categorie': 'Course E',
   'typePiste': '',
   'conditionsEngagement': "Pour 6 ans, n'ayant pas gagné 135.000. - Recul de 25 m. à 76.000.<br />",
   'hasTracking': False,
   'hasVideoHeat': False,
   'statut': 16,
   'classement': '16 - 15 - 10 - 5 - 6',
   'linkPrix': 'https://www.letrot.com/stats/fiche-course/2018-11-01/5305/1/resultats/arrivee-definitive',
   'replay': '<a href="https://www.letrot.com/fr/replay-courses/2018-11-01/5305/1" class="btn" target="_blank"><svg width="8px" height="12px" viewBox="0 0 8 12" version="1.1" xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink"><g id="Page-1" stroke="none" stroke-width="1" fill="none" fill-rule="evenodd"><polygon id="triangle" fill="#ffffff" transform="translate(4.000000, 6.000000) rotat

In [72]:
def courses(reunions):
    courses_list = []
    
    for i, row in reunions.iterrows():
        url = f"https://www.letrot.com/{row['lien']}/json"
        r = requests.get(url, headers=headers)
        courses = r.json()
        for c in courses["course"]:
            courses_list.append({"hippodrome": courses["nomHippodrome"],"date": row["date"], "idHippo": row["idHippo"],**c})
    return pd.DataFrame(courses_list)

In [58]:
for i,row in reu.iterrows():
    print(row["lien"])

/stats/courses/programme/2021-11-01/5305
/stats/courses/programme/2021-11-01/5901
/stats/courses/programme/2021-11-01/1601
/stats/courses/programme/2021-11-01/8201
/stats/courses/programme/2021-11-01/5006
/stats/courses/programme/2021-11-01/1407
/stats/courses/programme/2021-11-01/6902
/stats/courses/programme/2021-11-01/3001
/stats/courses/programme/2021-11-01/4417
/stats/courses/programme/2021-11-02/7500
/stats/courses/programme/2021-11-02/8301
/stats/courses/programme/2021-11-03/7500
/stats/courses/programme/2021-11-04/8201
/stats/courses/programme/2021-11-04/1302
/stats/courses/programme/2021-11-05/5901
/stats/courses/programme/2021-11-05/7500
/stats/courses/programme/2021-11-05/1304
/stats/courses/programme/2021-11-06/1702
/stats/courses/programme/2021-11-07/6102
/stats/courses/programme/2021-11-07/5008
/stats/courses/programme/2021-11-07/3102
/stats/courses/programme/2021-11-07/4411
/stats/courses/programme/2021-11-07/5307
/stats/courses/programme/2021-11-07/7104
/stats/courses/p

In [73]:
courses(reu)

,hippodrome,date,idHippo,heureCourse,discipline,numCourse,prix,allocation,distance,categorie,typePiste,conditionsEngagement,hasTracking,hasVideoHeat,statut,classement,linkPrix,replay
0,LAVAL,2022-11-01,5305,13:58,Attelé,1,PRIX PARIS-TURF,20 000,2 850,Course F,,Course Nationale<br />LeTROT Open des Régions ...,False,False,16,1 - 4 - 2 - 7 - 6,https://www.letrot.com/stats/fiche-course/2022...,"<a href=""https://www.letrot.com/fr/replay-cour..."
1,LAVAL,2022-11-01,5305,14:33,Attelé,2,PRIX MOET & CHANDON,20 000,2 850,Course F,,Course Nationale<br />LeTROT Open des Régions ...,False,True,16,10 - 1 - 14 - 15 - 11,https://www.letrot.com/stats/fiche-course/2022...,"<a href=""https://www.letrot.com/fr/replay-cour..."
2,LAVAL,2022-11-01,5305,15:15,Attelé,3,GRAND PRIX DE LA VILLE DE LAVAL,70 000,2 850,Course A,,"Course Européenne<br />Pour 5 à 10 ans inclus,...",False,True,16,7 - 1 - 6 - 11 - 8,https://www.letrot.com/stats/fiche-course/2022...,"<a href=""https://www.letrot.com/fr/replay-cour..."
3,LAVAL,2022-11-01,5305,15:52,Attelé,4,PRIX ANDRE-LOUIS DREUX,30 000,2 850,Course C,,Course Nationale<br />LeTROT Open des Régions ...,False,True,16,15 - 12 - 6 - 16 - 2,https://www.letrot.com/stats/fiche-course/2022...,"<a href=""https://www.letrot.com/fr/replay-cour..."
4,LAVAL,2022-11-01,5305,16:26,Attelé,5,PRIX CREDIT AGRICOLE ANJOU MAINE,27 000,2 850,Course E,,"Pour 6 ans, n'ayant pas gagné 135.000. - Recul...",False,True,16,5 - 4 - 8 - 14 - 2,https://www.letrot.com/stats/fiche-course/2022...,"<a href=""https://www.letrot.com/fr/replay-cour..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,VINCENNES,2022-11-04,7500,19:33,Monté,3,PRIX LOUIS TILLAYE,120 000,2 700,Groupe II,Grande piste,"Pour 3 ans, hongres exclus, ayant gagné au moi...",True,False,0,,https://www.letrot.com/stats/fiche-course/2022...,
96,VINCENNES,2022-11-04,7500,20:15,Attelé,4,PRIX BARBARA,53 000,2 100,Course D,Grande piste,"Départ à l'autostart<br />Pour 7 et 8 ans, n'a...",True,False,0,,https://www.letrot.com/stats/fiche-course/2022...,
97,VINCENNES,2022-11-04,7500,21:02,Attelé,5,PRIX ANGELINA,39 000,2 850,Course D,Petite piste,"Pour pouliches de 4 ans, n'ayant pas gagné 71....",True,False,0,,https://www.letrot.com/stats/fiche-course/2022...,
98,VINCENNES,2022-11-04,7500,21:42,Attelé,6,PRIX POSIDONIUS,33 000,2 100,Course E,Grande petite piste,Départ à l'autostart<br />Pour poulains entier...,True,False,0,,https://www.letrot.com/stats/fiche-course/2022...,


In [ ]:
def partants(course):
    partants = []
    for i, row in course.iterrows():
        url = f"https://www.letrot.com/stats/fiche-course/{row['date']}/{row['idHippo']}/{row['numCourse']}/partants/tableau"
        r = requests.get(url, headers=headers)
        soup = bs(r.text, "html.parser")
        table = soup.find("table", {"id": "result_table"})
        rows = table.find_all("tr", {"class": "even"})
        
        for row in rows:
            infos = table.find("tbody").find_all("tr")[0].select("td")
            
            num = infos.find("")
            
        

In [74]:
url = "https://www.letrot.com/stats/fiche-course/2018-11-01/5305/1/partants/tableau"
r = requests.get(url, headers=headers)
soup = bs(r.text, "html.parser")
table = soup.find("table", {"id": "result_table"})

In [78]:
rows = table.find_all("tr", {"class": "even"})

In [104]:
table.find("tbody").find_all("tr")[0].select("td")[0].select("span.bold")

[<span class="bold">NP</span>]